In [150]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [151]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [152]:
batch_size = 32  # Batch size for training.
epochs = 200  # Number of epochs to train for.
latent_dim = 260  # Latent dimensionality of the encoding space.
num_samples =12000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'C:/Users/user/Desktop/IVY WORK BOOK/PYTHON/Python ML notebooks/french_to_english.txt'

In [153]:
input_texts = []
target_texts =[]
input_characters = set()
target_characters = set()

In [155]:
lines =open(data_path, 'r', encoding='utf-8').read().split("\n")
for line in lines[: min(num_samples, len(lines)-1)]:
    input_text,target_text= line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [156]:
for line in lines[: min(num_samples, len(lines)-1)]:
      print(line)

Go.	Va !
Run!	Cours !
Run!	Courez !
Wow!	Ça alors !
Fire!	Au feu !
Help!	À l'aide !
Jump.	Saute.
Stop!	Ça suffit !
Stop!	Stop !
Stop!	Arrête-toi !
Wait!	Attends !
Wait!	Attendez !
I see.	Je comprends.
I try.	J'essaye.
I won!	J'ai gagné !
I won!	Je l'ai emporté !
Oh no!	Oh non !
Attack!	Attaque !
Attack!	Attaquez !
Cheers!	Santé !
Cheers!	À votre santé !
Cheers!	Merci !
Cheers!	Tchin-tchin !
Get up.	Lève-toi.
Got it!	J'ai pigé !
Got it!	Compris !
Got it?	Pigé ?
Got it?	Compris ?
Got it?	T'as capté ?
Hop in.	Monte.
Hop in.	Montez.
Hug me.	Serre-moi dans tes bras !
Hug me.	Serrez-moi dans vos bras !
I fell.	Je suis tombée.
I fell.	Je suis tombé.
I know.	Je sais.
I left.	Je suis parti.
I left.	Je suis partie.
I lost.	J'ai perdu.
I'm 19.	J'ai 19 ans.
I'm OK.	Je vais bien.
I'm OK.	Ça va.
Listen.	Écoutez !
No way!	C'est pas possible !
No way!	Impossible !
No way!	En aucun cas.
No way!	C'est hors de question !
No way!	Il n'en est pas question !
No way!	C'est exclu !
No way!	En aucune manière !

It's right.	C'est juste.
It's safer.	C'est plus sûr.
It's sweet.	C'est mignon.
It's sweet.	C'est gentil.
It's weird.	C'est bizarre.
It's wrong.	C'est faux.
It's yours.	C'est le tien.
It's yours.	C'est la tienne.
It's yours.	C'est la vôtre.
It's yours.	C'est le vôtre.
Just relax.	Détendez-vous.
Just relax.	Détends-toi.
Keep quiet!	Reste tranquille !
Keep quiet!	Restez tranquille !
Keep quiet.	Reste tranquille.
Keep quiet.	Restez tranquille.
Keep quiet.	Restez tranquilles.
Let him go!	Laisse-le partir !
Let him go!	Laissez-le partir !
Let him go!	Laisse-le s'en aller!
Let him go!	Laissez-le s'en aller!
Let him go.	Laisse-le partir !
Let him go.	Laissez-le partir !
Let him go.	Laisse-le s'en aller!
Let him go.	Laissez-le s'en aller!
Let him in.	Fais-le entrer.
Let him in.	Laisse-le entrer.
Let him in.	Laissez-le entrer.
Let him in.	Faites-le entrer.
Let me die.	Laissez-moi mourir.
Let me die.	Laisse-moi mourir.
Let me out!	Laisse-moi sortir !
Let me out!	Laissez-moi sortir !
Let me pay.	L

I like music.	J'aime la musique.
I like opera.	J'apprécie l'opéra.
I like sushi.	J'aime les sushis.
I like these.	J'aime ceux-ci.
I like these.	J'aime celles-ci.
I like women.	J'apprécie les femmes.
I like women.	J'aime les femmes.
I liked that.	J'ai aimé ça.
I love books.	J'adore les livres.
I love bread.	J'adore le pain.
I love games.	J'adore les jeux.
I love jokes.	J'adore les blagues.
I love jokes.	J'adore les plaisanteries.
I love music.	J'adore la musique.
I love music.	J'aime beaucoup la musique.
I love trips.	J'aime les voyages.
I loved that.	J'adorais ça.
I loved that.	J'ai adoré ça.
I made a bet.	J'ai effectué un pari.
I made plans.	J'ai élaboré des plans.
I made plans.	J'ai fait des projets.
I made these.	J'ai fait ceux-ci.
I made these.	J'ai fait celles-ci.
I made these.	J'ai confectionné ceux-ci.
I might stay.	Il se pourrait que je reste.
I missed you.	Tu m'as manqué.
I must hurry.	Je dois me grouiller.
I must hurry.	Je dois me dépêcher.
I must hurry.	Je dois me hâter.
I m

I wore a mask.	J'ai porté un masque.
I work nights.	Je travaille de nuit.
I worked hard.	Je travaillais dur.
I write poems.	J'écris des poèmes.
I'd better go.	Je ferais mieux d'y aller.
I'll allow it.	Je le permettrai.
I'll be brief.	Je serai brève.
I'll be brief.	Je serai bref.
I'll be going.	J'irai.
I'll be ready.	Je serai prêt.
I'll be ready.	Je serai prête.
I'll be there.	J'y serai.
I'll buy that.	J'achèterai ça.
I'll buy this.	Je vais acheter celui-ci.
I'll call him.	Je vais l'appeler au téléphone.
I'll call him.	Je l'appellerai.
I'll call you.	Je vous appellerai.
I'll eat here.	Je mangerai ici.
I'll find you.	Je te trouverai.
I'll get help.	J'obtiendrai de l'aide.
I'll get lost.	Je me perdrai.
I'll get some.	J'irai en chercher.
I'll get them.	J'irai les chercher.
I'll get this.	Je vais prendre ceci.
I'll go ahead.	J'irai en avant.
I'll go ahead.	Je continuerai.
I'll go check.	J'irai vérifier.
I'll go first.	J'irai en premier.
I'll go first.	Je vais y aller en premier.
I'll go fir

Happy New Year!	Bonne année !
Happy New Year!	Joyeuse année !
Happy birthday!	Bon anniversaire !
Have you eaten?	Tu as mangé ?
Have you eaten?	Avez-vous mangé ?
Have you slept?	As-tu dormi ?
Have you slept?	Avez-vous dormi ?
He almost died.	Il a failli mourir.
He also saw it.	Il l’a aussi vu.
He asked me to.	Il me l'a demandé.
He became rich.	Il est devenu riche.
He bit his lip.	Il se mordit la lèvre.
He bit his lip.	Il s'est mordu la lèvre.
He came by bus.	Il est venu en bus.
He came by bus.	Il vint en bus.
He came by car.	Il vint en voiture.
He came by car.	Il est venu en voiture.
He can't count.	Il ne sait pas compter.
He can't do it.	Il est incapable de le faire.
He can't do it.	Il n'est pas capable de le faire.
He can't drive.	Il ne sait pas conduire.
He can't drive.	Il ne peut pas conduire.
He cannot swim.	Il ne sait pas nager.
He could do it.	Lui pourrait le faire.
He counts fast.	Il compte vite.
He deceived me.	Il m'a trompé.
He deserved it.	Il l'a mérité.
He dislikes me.	Il ne

We're not sure.	Nous n'en sommes pas sûres.
We're obedient.	Nous sommes obéissants.
We're obedient.	Nous sommes obéissantes.
We're partners.	Nous sommes partenaires.
We're powerful.	Nous sommes puissants.
We're powerful.	Nous sommes puissantes.
We're prepared.	Nous sommes prêtes.
We're punctual.	Nous sommes ponctuels.
We're punctual.	Nous sommes ponctuelles.
We're quitting.	Nous abandonnons.
We're reliable.	Nous sommes fiables.
We're retiring.	Nous partons à la retraite.
We're retiring.	Nous partons à la pension.
We're ruthless.	Nous sommes impitoyables.
We're safe now.	Nous sommes désormais en sécurité.
We're so sorry.	Nous sommes tellement désolés.
We're standing.	Nous sommes debout.
We're starving.	Nous mourons de faim.
We're stubborn.	Nous sommes têtus.
We're stubborn.	Nous sommes têtues.
We're students.	Nous sommes étudiants.
We're the best.	Nous sommes les meilleurs.
We're the best.	Nous sommes les meilleures.
We're the last.	Nous sommes les derniers.
We're the last.	Nous sommes 

I've had enough.	Ça suffit.
I've had enough.	J'en ai assez entendu.
I've had enough.	J'en ai assez vu.
I've had enough.	J'en ai assez avalé.
I've had enough.	J'en ai assez gobé.
I've had enough.	J'en ai assez supporté.
I've had enough.	J'en ai par-dessus la tête.
I've had enough.	Je suis gavé.
Is Tom a member?	Tom est-il membre ?
Is Tom an actor?	Tom est-il acteur ?
Is Tom dreaming?	Tom est-il en train de rêver ?
Is anybody here?	Il y a quelqu'un ?
Is anybody home?	Quelqu'un est-il là ?
Is anybody home?	Y a-t-il quelqu'un ?
Is anybody home?	Y a-t-il quelqu'un dans la maison ?
Is anybody hurt?	Quiconque est-il blessé ?
Is anyone there?	Quiconque est-il là ?
Is everybody OK?	Tout le monde va-t-il bien ?
Is he a teacher?	Est-il enseignant ?
Is he all right?	Va-t-il bien ?
Is he all right?	Se porte-t-il bien ?
Is he breathing?	Respire-t-il ?
Is it all there?	Tout est-il là ?
Is it all wrong?	Est-ce tout faux ?
Is it all wrong?	Est-ce entièrement mal ?
Is it clear now?	Est-ce désormais clai

Drinks are on me.	Les boissons sont pour moi.
Drinks are on me.	Les verres sont sur mon compte.
Drinks are on me.	Les verres sont pour moi.
Eat meals slowly.	Mange tes repas lentement.
Eat your spinach.	Mange tes épinards !
Eat your spinach.	Mangez vos épinards !
Eat your veggies.	Mange tes légumes !
Eat your veggies.	Mangez vos légumes !
Enjoy yourselves.	Amusez-vous bien !
Everybody groans.	Tout le monde râle.
Everybody saw it.	Tout le monde l'a vu.
Everybody saw it.	Tout le monde le vit.
Everybody's dead.	Tout le monde est mort.
Everybody's dead.	Tout le monde a cané.
Everybody's dead.	Tout le monde a crevé.
Everybody's here.	Tout le monde est là.
Everybody's home.	Tout le monde est chez lui.
Everybody's safe.	Tout le monde est en sécurité.
Everyone changes.	Tout le monde change.
Everyone cheered.	Tout le monde a applaudi.
Everyone cheered.	Tout le monde applaudit.
Everyone had fun.	Tout le monde s'est amusé.
Everyone is here.	Tout le monde est là.
Everyone laughed.	Tout le monde ri

In [157]:
print(input_texts[0:10])
print(target_texts[0:10])


['Go.', 'Run!', 'Run!', 'Wow!', 'Fire!', 'Help!', 'Jump.', 'Stop!', 'Stop!', 'Stop!']
['\tVa !\n', '\tCours\u202f!\n', '\tCourez\u202f!\n', '\tÇa alors\u202f!\n', '\tAu feu !\n', "\tÀ l'aide\u202f!\n", '\tSaute.\n', '\tÇa suffit\u202f!\n', '\tStop\u202f!\n', '\tArrête-toi !\n']


In [158]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [159]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 12000
Number of unique input tokens: 73
Number of unique output tokens: 94
Max sequence length for inputs: 17
Max sequence length for outputs: 59


In [160]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [161]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '$': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '0': 12,
 '1': 13,
 '5': 14,
 '6': 15,
 '9': 16,
 ':': 17,
 '?': 18,
 'A': 19,
 'B': 20,
 'C': 21,
 'D': 22,
 'E': 23,
 'F': 24,
 'G': 25,
 'H': 26,
 'I': 27,
 'J': 28,
 'K': 29,
 'L': 30,
 'M': 31,
 'N': 32,
 'O': 33,
 'P': 34,
 'Q': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'Y': 41,
 'Z': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68,
 '\xa0': 69,
 '«': 70,
 '»': 71,
 'À': 72,
 'Ç': 73,
 'É': 74,
 'Ê': 75,
 'Ô': 76,
 'à': 77,
 'â': 78,
 'ç': 79,
 'è': 80,
 'é': 81,
 'ê': 82,
 'ë': 83,
 'î': 84,
 'ï': 85,
 'ô': 86,
 'ù': 87,
 'û': 88,
 'œ': 89,
 '\u2009': 90,
 '‘': 91,
 '’': 92,
 '\u202f': 93}

In [162]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [163]:
print(encoder_input_data[0:1])

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


In [164]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]]=1.
    for t, char in enumerate(target_text):
         decoder_input_data[i, t, target_token_index[char]]=1.
    if t > 0:
         decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [165]:
encoder_input_data[0].shape

(17, 73)

In [166]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [167]:
print(encoder_states[0].shape)
print(encoder_outputs)

(None, 260)
Tensor("lstm_5/strided_slice_18:0", shape=(None, 260), dtype=float32)


In [168]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [169]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, None, 73)     0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, None, 94)     0                                            
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, 260), (None, 347360      input_15[0][0]                   
__________________________________________________________________________________________________
lstm_6 (LSTM)                   [(None, None, 260),  369200      input_16[0][0]                   
                                                                 lstm_5[0][1]              

In [186]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=10,
          validation_split=0.2)

Train on 9600 samples, validate on 2400 samples
Epoch 1/10
9600/9600 [==============================] - 138s 14ms/step - loss: 2.0433e-09 - accuracy: 0.0169 - val_loss: 2.0205e-09 - val_accuracy: 0.0169
Epoch 2/10
9600/9600 [==============================] - 138s 14ms/step - loss: 2.0427e-09 - accuracy: 0.0169 - val_loss: 2.0205e-09 - val_accuracy: 0.0169
Epoch 3/10
9600/9600 [==============================] - 136s 14ms/step - loss: 2.0425e-09 - accuracy: 0.0169 - val_loss: 2.0205e-09 - val_accuracy: 0.0169
Epoch 4/10
9600/9600 [==============================] - 140s 15ms/step - loss: 2.0422e-09 - accuracy: 0.0169 - val_loss: 2.0205e-09 - val_accuracy: 0.0169
Epoch 5/10
9600/9600 [==============================] - 131s 14ms/step - loss: 2.0418e-09 - accuracy: 0.0169 - val_loss: 2.0205e-09 - val_accuracy: 0.0169
Epoch 6/10
9600/9600 [==============================] - 137s 14ms/step - loss: 2.0413e-09 - accuracy: 0.0169 - val_loss: 2.0205e-09 - val_accuracy: 0.0169
Epoch 7/10
9600/9600 [

In [187]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)


In [188]:
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)


In [189]:
print(decoder_model.get_layer)

<bound method Network.get_layer of <keras.engine.training.Model object at 0x000001C65155D9C8>>


In [190]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [191]:
print(reverse_input_char_index)

{0: ' ', 1: '!', 2: '"', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '0', 10: '1', 11: '2', 12: '3', 13: '4', 14: '5', 15: '6', 16: '7', 17: '8', 18: '9', 19: ':', 20: '?', 21: 'A', 22: 'B', 23: 'C', 24: 'D', 25: 'E', 26: 'F', 27: 'G', 28: 'H', 29: 'I', 30: 'J', 31: 'K', 32: 'L', 33: 'M', 34: 'N', 35: 'O', 36: 'P', 37: 'Q', 38: 'R', 39: 'S', 40: 'T', 41: 'U', 42: 'V', 43: 'W', 44: 'Y', 45: 'Z', 46: 'a', 47: 'b', 48: 'c', 49: 'd', 50: 'e', 51: 'f', 52: 'g', 53: 'h', 54: 'i', 55: 'j', 56: 'k', 57: 'l', 58: 'm', 59: 'n', 60: 'o', 61: 'p', 62: 'q', 63: 'r', 64: 's', 65: 't', 66: 'u', 67: 'v', 68: 'w', 69: 'x', 70: 'y', 71: 'z', 72: '’'}


In [192]:
print(reverse_target_char_index)

{0: '\t', 1: '\n', 2: ' ', 3: '!', 4: '$', 5: '&', 6: "'", 7: '(', 8: ')', 9: ',', 10: '-', 11: '.', 12: '0', 13: '1', 14: '5', 15: '6', 16: '9', 17: ':', 18: '?', 19: 'A', 20: 'B', 21: 'C', 22: 'D', 23: 'E', 24: 'F', 25: 'G', 26: 'H', 27: 'I', 28: 'J', 29: 'K', 30: 'L', 31: 'M', 32: 'N', 33: 'O', 34: 'P', 35: 'Q', 36: 'R', 37: 'S', 38: 'T', 39: 'U', 40: 'V', 41: 'Y', 42: 'Z', 43: 'a', 44: 'b', 45: 'c', 46: 'd', 47: 'e', 48: 'f', 49: 'g', 50: 'h', 51: 'i', 52: 'j', 53: 'k', 54: 'l', 55: 'm', 56: 'n', 57: 'o', 58: 'p', 59: 'q', 60: 'r', 61: 's', 62: 't', 63: 'u', 64: 'v', 65: 'w', 66: 'x', 67: 'y', 68: 'z', 69: '\xa0', 70: '«', 71: '»', 72: 'À', 73: 'Ç', 74: 'É', 75: 'Ê', 76: 'Ô', 77: 'à', 78: 'â', 79: 'ç', 80: 'è', 81: 'é', 82: 'ê', 83: 'ë', 84: 'î', 85: 'ï', 86: 'ô', 87: 'ù', 88: 'û', 89: 'œ', 90: '\u2009', 91: '‘', 92: '’', 93: '\u202f'}


In [193]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    print(states_value)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.0
    stop_condition = False
    decoded_sentence = ' '
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        print(output_tokens)
        sampled_token_index = np.argmax(output_tokens[0,-1, :])
        #print(output_tokens[0,-1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]
    return decoded_sentence

In [194]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index:seq_index + 1]
    decoded_sentence1 = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence1)

[array([[ 0.5210626 ,  0.5505711 , -0.43197793,  0.26270506, -0.0427707 ,
        -0.34271902, -0.5158343 , -0.40989152, -0.16335925, -0.03285475,
         0.1876498 , -0.23572536,  0.3300564 , -0.35566503,  0.41091117,
         0.20821765, -0.2776293 ,  0.4278048 ,  0.46354392,  0.3216448 ,
        -0.14837787,  0.51688904, -0.36207625, -0.02484701,  0.46228737,
         0.42386547,  0.30320507, -0.05162446,  0.48183522,  0.38209394,
        -0.51194286,  0.40888256, -0.12038267, -0.31047276, -0.32620052,
         0.5464249 , -0.37892902, -0.35873738,  0.3978033 ,  0.26688516,
         0.3848327 , -0.2653699 ,  0.21783073, -0.2984246 , -0.3972601 ,
         0.49081624,  0.13984075, -0.5156971 ,  0.5121596 ,  0.531774  ,
        -0.36232293, -0.33332294, -0.12857659, -0.47833523,  0.33360007,
        -0.04274077,  0.39632896,  0.4314502 , -0.3060831 , -0.4945344 ,
         0.2864569 ,  0.19473948, -0.33764738,  0.26849985,  0.5110668 ,
        -0.4706129 ,  0.21606334, -0.4792647 , -0.

[array([[ 0.51925606,  0.54986644, -0.43126857,  0.26166663, -0.04325034,
        -0.3408864 , -0.5140636 , -0.40836117, -0.16387203, -0.03408232,
         0.18535005, -0.2331865 ,  0.32959726, -0.35457358,  0.40881294,
         0.20463452, -0.27532223,  0.42536438,  0.46187937,  0.31509617,
        -0.14641608,  0.5158968 , -0.36192584, -0.02443616,  0.45927492,
         0.4229294 ,  0.30354768, -0.05159901,  0.47978407,  0.37773627,
        -0.5105616 ,  0.40775535, -0.11920556, -0.31077805, -0.32637307,
         0.5437203 , -0.3775822 , -0.35713547,  0.39652804,  0.26594615,
         0.38480645, -0.2640892 ,  0.21799402, -0.29689655, -0.3945153 ,
         0.48877972,  0.13788714, -0.51468056,  0.5102122 ,  0.5308016 ,
        -0.36116427, -0.33162796, -0.12655613, -0.4783957 ,  0.33285907,
        -0.04287511,  0.39466345,  0.4296398 , -0.30244926, -0.4925633 ,
         0.28505477,  0.19255729, -0.3336374 ,  0.26588222,  0.5091579 ,
        -0.4689292 ,  0.21633732, -0.47698385, -0.

   2.0933636e-03 2.3094020e-03 7.6697790e-04 2.1992619e-03]]]
-
Input sentence: I see.
Decoded sentence:  

[array([[ 0.5466937 ,  0.5783238 , -0.4602197 ,  0.27834436, -0.04720819,
        -0.36835378, -0.54116106, -0.43177938, -0.18637303, -0.04067443,
         0.2112072 , -0.25776258,  0.35553777, -0.37111875,  0.43498215,
         0.23204875, -0.29716924,  0.45224002,  0.49359146,  0.35036024,
        -0.16512468,  0.54267746, -0.38621947, -0.02584979,  0.49209505,
         0.44531947,  0.32663193, -0.05423122,  0.5135639 ,  0.4049357 ,
        -0.5422851 ,  0.4282105 , -0.13028988, -0.33585745, -0.3621988 ,
         0.58405703, -0.41088232, -0.38173947,  0.42894036,  0.28426892,
         0.40808722, -0.28751788,  0.23794216, -0.32935697, -0.42198032,
         0.5153051 ,  0.15429066, -0.5423022 ,  0.54897803,  0.5613605 ,
        -0.38915417, -0.36485484, -0.14091612, -0.50732535,  0.35909918,
        -0.04462214,  0.42030895,  0.4693155 , -0.33341423, -0.5205389 ,
         0.3117

[[[0.00122249 0.8158376  0.00212923 0.00185241 0.00550038 0.00111531
   0.0015719  0.00131999 0.00162441 0.00296291 0.00107816 0.00292692
   0.00229098 0.0008542  0.00180767 0.00157619 0.00190374 0.00092969
   0.0028193  0.00176998 0.00115183 0.00188528 0.00292714 0.00254508
   0.00187274 0.00145778 0.00084317 0.00136686 0.00110745 0.00182038
   0.00121527 0.00265348 0.0020584  0.00131908 0.00408308 0.00250358
   0.00133875 0.00183817 0.00166418 0.00329944 0.00188938 0.00207474
   0.0015797  0.00139661 0.00285136 0.00261859 0.00088429 0.0055776
   0.00086897 0.00337223 0.00280019 0.00307443 0.00089401 0.00193721
   0.00103969 0.00223211 0.00130199 0.00215042 0.00163764 0.00085316
   0.00124138 0.00255649 0.00280525 0.00145656 0.00195446 0.00196066
   0.00113072 0.00421379 0.00289895 0.00157175 0.00359897 0.00270682
   0.00157597 0.00192439 0.00260193 0.00225209 0.00154943 0.00183325
   0.00256904 0.00138902 0.00105103 0.00156857 0.00118644 0.00087616
   0.00176614 0.00124148 0.00161027

[[[0.00123037 0.81462836 0.00213939 0.00186427 0.00553819 0.00112319
   0.00157696 0.00133011 0.00163231 0.00298369 0.00108428 0.00295428
   0.00230613 0.00085931 0.00181775 0.0015856  0.00191416 0.00093675
   0.00283187 0.00178133 0.00115866 0.00189863 0.00294169 0.00255988
   0.00188161 0.00146628 0.00084855 0.00137513 0.00111508 0.00183451
   0.00122323 0.00267191 0.00206729 0.00132686 0.00411182 0.00252668
   0.00134774 0.00184996 0.00167497 0.00332194 0.00189817 0.00208918
   0.00158974 0.00140713 0.00286975 0.0026373  0.00089091 0.00562044
   0.00087426 0.00339426 0.00282088 0.00309232 0.00090201 0.00195482
   0.0010465  0.00223963 0.00130759 0.00216579 0.00164978 0.00085813
   0.00124695 0.00257263 0.00282367 0.00146834 0.00196256 0.00197575
   0.00113449 0.00423978 0.00291694 0.00158278 0.00362568 0.00272417
   0.00158795 0.00193297 0.0026199  0.00227264 0.00156059 0.00184142
   0.00258302 0.00140003 0.00105965 0.00157872 0.00119107 0.00088056
   0.0017821  0.00125282 0.0016217

...

   0.0023953  0.0026092  0.0009142  0.00253168]]]
-
Input sentence: Really?
Decoded sentence:  

[array([[ 0.5402142 ,  0.57264316, -0.45148104,  0.27391416, -0.04492424,
        -0.36495373, -0.53553617, -0.4261239 , -0.1773092 , -0.03881555,
         0.20550151, -0.25162953,  0.3510057 , -0.36636412,  0.42911848,
         0.22821768, -0.29158306,  0.44604838,  0.4868743 ,  0.3393486 ,
        -0.15986905,  0.53677344, -0.37914085, -0.02610048,  0.48499832,
         0.43979648,  0.3211382 , -0.05318172,  0.5054772 ,  0.39672238,
        -0.5350205 ,  0.42371064, -0.12532657, -0.32770905, -0.35322475,
         0.57585216, -0.40318483, -0.3776226 ,  0.42027336,  0.2802949 ,
         0.40370607, -0.27948123,  0.23599222, -0.3235564 , -0.41618422,
         0.5083958 ,  0.14972162, -0.53571224,  0.5417041 ,  0.5538121 ,
        -0.38297024, -0.3589962 , -0.1343907 , -0.5018401 ,  0.3515846 ,
        -0.04490108,  0.41637796,  0.46218526, -0.3253527 , -0.51400274,
         0.305982  ,  0.20

      dtype=float32)]
[[[1.1477447e-03 8.2526815e-01 2.0159094e-03 1.7514249e-03 5.2791699e-03
   1.0454346e-03 1.4853940e-03 1.2398722e-03 1.5358594e-03 2.8174105e-03
   1.0097929e-03 2.7963209e-03 2.1696570e-03 8.0018205e-04 1.7096731e-03
   1.4907299e-03 1.7889244e-03 8.6936424e-04 2.6921120e-03 1.6651575e-03
   1.0834699e-03 1.7850797e-03 2.7808328e-03 2.4266676e-03 1.7754118e-03
   1.3729071e-03 7.8612153e-04 1.2881269e-03 1.0391570e-03 1.7258164e-03
   1.1457242e-03 2.5276658e-03 1.9508116e-03 1.2415471e-03 3.8960048e-03
   2.3775506e-03 1.2588036e-03 1.7446724e-03 1.5712313e-03 3.1616276e-03
   1.7859065e-03 1.9683999e-03 1.4949789e-03 1.3213636e-03 2.7208382e-03
   2.4987811e-03 8.2527165e-04 5.3818906e-03 8.1515295e-04 3.2134608e-03
   2.6681323e-03 2.9254486e-03 8.3777268e-04 1.8470432e-03 9.8131085e-04
   2.1143379e-03 1.2249979e-03 2.0195490e-03 1.5538326e-03 7.9469354e-04
   1.1644274e-03 2.4323466e-03 2.6553003e-03 1.3674776e-03 1.8497568e-03
   1.8542581e-03 1.0629887e-0

Decoded sentence:  

[array([[ 0.55474955,  0.5870087 , -0.4695323 ,  0.28167027, -0.05267134,
        -0.37770268, -0.5484549 , -0.4392575 , -0.19533809, -0.04456383,
         0.21729723, -0.26248774,  0.36428586, -0.37547493,  0.4421792 ,
         0.23969558, -0.30162776,  0.4583702 ,  0.50227994,  0.35468382,
        -0.16973408,  0.5508214 , -0.3931356 , -0.02437311,  0.5015602 ,
         0.4512786 ,  0.3364384 , -0.05553021,  0.5228911 ,  0.4099486 ,
        -0.5506056 ,  0.4334548 , -0.13368097, -0.34388483, -0.37177423,
         0.5945545 , -0.41775373, -0.3897991 ,  0.43839732,  0.29226553,
         0.41485745, -0.2935434 ,  0.24511991, -0.3404645 , -0.4294629 ,
         0.52148676,  0.16251986, -0.54948145,  0.5623219 ,  0.56970423,
        -0.39866802, -0.37645876, -0.1431028 , -0.5150151 ,  0.3634074 ,
        -0.04749152,  0.4305014 ,  0.48077914, -0.34172574, -0.52684903,
         0.31911898,  0.21566375, -0.37264678,  0.28649577,  0.5454244 ,
        -0.49407548,  0.24288

      dtype=float32)]
[[[1.0526573e-03 8.3676177e-01 1.8859153e-03 1.6186704e-03 5.0368668e-03
   9.5707708e-04 1.3816698e-03 1.1361751e-03 1.4255599e-03 2.6584954e-03
   9.2727313e-04 2.6309432e-03 2.0186969e-03 7.3311559e-04 1.5917921e-03
   1.3804910e-03 1.6575744e-03 7.9224102e-04 2.5357236e-03 1.5391783e-03
   9.9548080e-04 1.6570466e-03 2.6149203e-03 2.2861378e-03 1.6525220e-03
   1.2747271e-03 7.1587891e-04 1.1902250e-03 9.5359446e-04 1.6128580e-03
   1.0611516e-03 2.3605945e-03 1.8192998e-03 1.1403137e-03 3.6839207e-03
   2.2332049e-03 1.1573230e-03 1.6221922e-03 1.4614101e-03 2.9950803e-03
   1.6610502e-03 1.8354573e-03 1.3893811e-03 1.2282300e-03 2.5681686e-03
   2.3454933e-03 7.5269426e-04 5.1562283e-03 7.4907864e-04 3.0308089e-03
   2.5095849e-03 2.7619670e-03 7.6926773e-04 1.7391156e-03 9.0902939e-04
   1.9620901e-03 1.1281335e-03 1.8776306e-03 1.4498298e-03 7.2210078e-04
   1.0667816e-03 2.2858672e-03 2.4845733e-03 1.2685052e-03 1.7251991e-03
   1.7151267e-03 9.7683875e-0

      dtype=float32)]
[[[1.0746558e-03 8.3416861e-01 1.9156312e-03 1.6419472e-03 5.0894292e-03
   9.7510469e-04 1.4057183e-03 1.1583122e-03 1.4526910e-03 2.6986792e-03
   9.4853324e-04 2.6678212e-03 2.0520671e-03 7.4659765e-04 1.6170770e-03
   1.4051098e-03 1.6894065e-03 8.0646120e-04 2.5683530e-03 1.5654756e-03
   1.0149846e-03 1.6886645e-03 2.6559255e-03 2.3223138e-03 1.6776130e-03
   1.2964578e-03 7.3038507e-04 1.2147259e-03 9.7434607e-04 1.6415926e-03
   1.0803692e-03 2.3963277e-03 1.8508699e-03 1.1648547e-03 3.7435733e-03
   2.2661011e-03 1.1796156e-03 1.6486190e-03 1.4842920e-03 3.0382574e-03
   1.6959191e-03 1.8570228e-03 1.4130536e-03 1.2516974e-03 2.5982908e-03
   2.3784209e-03 7.6899363e-04 5.2053090e-03 7.6534506e-04 3.0776209e-03
   2.5418112e-03 2.7944969e-03 7.8745733e-04 1.7670561e-03 9.2599558e-04
   1.9941956e-03 1.1490241e-03 1.9170319e-03 1.4707053e-03 7.3810294e-04
   1.0872550e-03 2.3250959e-03 2.5230520e-03 1.2967441e-03 1.7534501e-03
   1.7415858e-03 9.9635299e-0

In [142]:
from __future__ import print_function